### Simple Gen AI app with langchain and langsmith

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

### scrap the data from a website

In [5]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
link ="https://en.wikipedia.org/wiki/List_of_Australia_national_cricket_captains"
doc  = WebBaseLoader(link)
docs =doc.load()
docs

[Document(metadata={'source': 'https://www.cricbuzz.com/live-cricket-scorecard/4297/eng-vs-aus-5th-test-australia-in-england-2005', 'title': 'Cricket scorecard - England vs Australia, 5th Test, Australia in England, 2005 | Cricbuzz.com', 'description': 'Catch live and fully detailed scorecard of England vs Australia, 5th Test, Sep 08, Australia in England, 2005 on Cricbuzz', 'language': 'en'}, page_content='\n Cricket scorecard - England vs Australia, 5th Test, Australia in England, 2005 | Cricbuzz.com  ✖Live ScoresScheduleArchivesNewsAll Stories  Premium Editorials Latest NewsTopicsSpotlightOpinionsSpecialsStats & AnalysisInterviewsLive BlogsHarsha BhogleSeries  Asia Cup 2025 South Africa tour of England, 2025 Caribbean Premier League 2025 The Hundred Mens Competition 2025 Australia Women tour of India, 2025 ICC Womens World Cup 2025 India tour of England, 2025 South Africa tour of Australia, 2025 The Hundred Womens Competition 2025 Duleep Trophy 2025 All Series »Teams   Test Teams In

In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter  =   RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_docs = splitter.split_documents(docs)
len(splitted_docs)

12

In [29]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

C:\Users\yasha\AppData\Local\Temp\ipykernel_32440\2785657573.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="mxbai-embed-large")


In [35]:
a = embeddings.embed_documents([d.page_content for d in splitted_docs])

In [45]:
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='mxbai-embed-large', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [46]:
from langchain_community.vectorstores import Chroma

# Create a Chroma database and store docs
db = Chroma.from_documents(
    splitted_docs,
    embeddings,
    persist_directory="./chroma_store"  # saves DB locally
)

# Save database to disk
db.persist()


C:\Users\yasha\AppData\Local\Temp\ipykernel_32440\3154922141.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [47]:
# Reload the DB later
db = Chroma(
    persist_directory="./chroma_store",
    embedding_function=embeddings
)

# Example query
query = "Who won the 5th test in 2005 Ashes series?"
results = db.similarity_search(query, k=3)

for r in results:
    print(r.page_content[:300])  # show first 300 chars


C:\Users\yasha\AppData\Local\Temp\ipykernel_32440\831728929.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


vs Norway Women3rd MatchDenmark Women vs Sweden Women4th Match    England vs Australia, 5th Test - Live Cricket Score, Commentary    Series:  Australia in England, 2005 Venue: Kennington Oval, London Date & Time: Sep 08-Sep 12, 10:30 AM LOCAL    Commentary Scorecard Squads Live Blog Match Facts News
38.3 3 124 6 0 1 3.24        Michael Clarke   2 0 6 0 0 0 3.00        Shaun Tait   5 0 28 1 0 1 5.60          Australia 2nd Innings 4-0 (0.4 Ov)   Batter  R B 4s 6s SR      Justin Langer    not out  0 4 0 0 0.00        Matthew Hayden    not out  0 0 0 0 0.00      Extras  4   (b 0, lb 4, w 0, nb 0, p
Cricket scorecard - England vs Australia, 5th Test, Australia in England, 2005 | Cricbuzz.com  ✖Live ScoresScheduleArchivesNewsAll Stories  Premium Editorials Latest NewsTopicsSpotlightOpinionsSpecialsStats & AnalysisInterviewsLive BlogsHarsha BhogleSeries  Asia Cup 2025 South Africa tour of England


In [52]:
from langchain_community.llms import Ollama

# Load Gemma3 4B model
llm = Ollama(model="gemma3:4b")
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful cricket expert. do not use the internet. Use only the provided context to answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer."),
    ("human", "{question}")
])
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
# Chain = prompt → llm → output_parser
chain = prompt | llm | output_parser
response = chain.invoke({"question": "who is the current captain of Australia cricket team?"})
print(response)



I do not have the information to tell you who the current captain of the Australia cricket team is.
